In [ ]:
import os
from dotenv import load_dotenv

# Get the current working directory
current_dir = os.getcwd()
# Get the parent directory (one level above the current working directory)
parent_dir = os.path.dirname(current_dir)
# Construct the path to the .env file in the parent directory
dotenv_path = os.path.join(parent_dir, ".env")
# Load environment variables from the .env file
_ = load_dotenv(dotenv_path)

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")


In [ ]:
# load document
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path="../documents/events.csv")
data = loader.load()
len(data)



In [ ]:
# split documents (not needed for single csv file as it is already split into lines)
# from langchain.text_splitter import RecursiveCharacterTextSplitter

# text_splitter = RecursiveCharacterTextSplitter(
# # Chunk size is new linewith separator set
#     # chunk_size =
#     chunk_overlap  = 2,
#     separators="\n"
# )
# docs = text_splitter.split_documents(data)
# print(len(data))
# print(len(docs))


In [ ]:
# embedding
import os
import sys
import openai
from dotenv import load_dotenv

# Get the current working directory
current_dir = os.getcwd()
# Get the parent directory (one level above the current working directory)
parent_dir = os.path.dirname(current_dir)
# Construct the path to the .env file in the parent directory
dotenv_path = os.path.join(parent_dir, ".env")
# Load environment variables from the .env file
_ = load_dotenv(dotenv_path)

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

sentence1 = "I like purple"
sentence2 = "I like pink"
sentence3 = "That's the sky"

embedding1 = embedding.embed_query(sentence1)
# # embedding2 = embedding.embed_query(sentence2)
# # embedding3 = embedding.embed_query(sentence3)

# print(embedding1)

# # import numpy as np
# # np.dot(embedding1, embedding2)


In [ ]:
# vector stores
